In [1]:
import json
import os

# download ffmpeg (https://www.ffmpeg.org/download.html) and set the path
ffmpeg_path = r"C:\Users\Sudheera\Documents\ffmpeg-n6.1-latest-win64-gpl-6.1\ffmpeg-n6.1-latest-win64-gpl-6.1\bin"
os.environ['PATH'] += os.pathsep + ffmpeg_path

with open('vars.json') as f:
    data = json.load(f)

## 1. Download subtitles from YouTube

In [2]:
import yt_dlp

# Function to download subtitles
def download_subtitles(url, lang='en'):
    ydl_opts = {
        'writesubtitles': True,          # Download subtitles
        'writeautomaticsub': True,       # Download automatic subtitles generated by YouTube
        'subtitleslangs': [lang],        # Subtitle language (default is English)
        'subtitlesformat': 'srt',        # Format for subtitles (e.g., srt, vtt)
        'skip_download': True,           # Do not download the video file itself
        'restrictfilenames': True,       # Do not allow special characters in file names
        'outtmpl': '%(title)s.%(ext)s',  # Output file naming
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [3]:
# URL of the YouTube video
video_url = 'https://www.youtube.com/watch?v=CDZ9REOh2xA'

# Download subtitle
download_subtitles(video_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=CDZ9REOh2xA
[youtube] CDZ9REOh2xA: Downloading webpage
[youtube] CDZ9REOh2xA: Downloading ios player API JSON
[youtube] CDZ9REOh2xA: Downloading web creator player API JSON
[youtube] CDZ9REOh2xA: Downloading m3u8 information
[info] CDZ9REOh2xA: Downloading subtitles: en


[info] CDZ9REOh2xA: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: Elon_Musk_s_approach_to_problem-solving_Lex_Fridman_Podcast.en.vtt
[download] Destination: Elon_Musk_s_approach_to_problem-solving_Lex_Fridman_Podcast.en.vtt
[download] 100% of   68.77KiB in 00:00:00 at 76.26KiB/s


## 2. Download audio file from Youtube and transcribe it using Google Speech to Text

In [6]:
import yt_dlp

def download_audio(youtube_url, output_file):
    ydl_opts = {
        'format': 'bestaudio/best',  # Select the best available audio quality
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3'  # You can change this to 'wav', 'm4a', etc.
        }],
        'outtmpl': output_file,  # Output file name and path
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
        
    return True

In [8]:
# Download Audio
download_audio(video_url, 'audio_new')

[youtube] Extracting URL: https://www.youtube.com/watch?v=CDZ9REOh2xA
[youtube] CDZ9REOh2xA: Downloading webpage
[youtube] CDZ9REOh2xA: Downloading ios player API JSON
[youtube] CDZ9REOh2xA: Downloading web creator player API JSON
[youtube] CDZ9REOh2xA: Downloading m3u8 information
[info] CDZ9REOh2xA: Downloading 1 format(s): 251
[download] Destination: audio_new
[download] 100% of    6.53MiB in 00:00:07 at 928.98KiB/s 
[ExtractAudio] Destination: audio_new.mp3
Deleting original file audio_new (pass -k to keep)


True

In [9]:
audio_file_path = r"audio_new.mp3"
    
import io
from pydub import AudioSegment

# Read the audio file into memory using io.BytesIO
with open(audio_file_path, 'rb') as audio_file:
    audio_data = io.BytesIO(audio_file.read())

    # Load the audio from the in-memory file
    audio = AudioSegment.from_file(audio_data, format="mp3")
    
    # Print the sampling rate
    print(f"Sampling Rate: {audio.frame_rate} Hz")

Sampling Rate: 48000 Hz


## 3. Transcribe the audio using Google Speech to Text

## 4. Pre-process downloaded transcript

In [ ]:
# After transcribing an audio file you can download a csv file and a Json file from Google Speech-to-text
# csv :- transcript, confidence, speaker tag, language code, start time, end time (some words can be wrong)
# json :- Additional information you have here is the confidence of each word

# first we will read files separately and create one dataframe with necessary data 

import pandas as pd
import json

csv_file = 'audio.csv'
json_file = 'audio.json'

csv_column_names = {
    'Start time': 'start_time',
    'End time': 'end_time',
    'Language code': 'language_code',
    'Confidence': 'confidence',
    'Channel': 'channel',
    'Speaker tag': 'speaker_tag',
    'Transcript': 'transcript'
}

# Json file format :- 
# {
#    "results": [
#        { "alternatives": [
#               {
#                   "confidence": _
#                   "transcript": _
#                   "words": [
#                       {
#                           "confidence": _,
#                           "endTime": _,
#                           "startTime": _,
#                           "word": _
#                       }, ...
#           ],
#           "languageCode": _,
#           "resultEndTime": _
#       }, ...
#       ...
#       ...
#       {                                           <- last item of the list (after all transcriptions)
#           "alternatives": {
#               "words": [
#                   {
#                      "confidence": _,
#                      "endTime": _,
#                      "speakerLabel": _,
#                      "speakerTag": _,
#                      "startTime": _,
#                      "word": _
#                   },....                          <- one item for each word
#               ]
#           }
#       }
#   ]
# }